# CAIRO: Robust Regression via Rank-then-Calibrate

This tutorial explains **CAIRO** (Calibrate After Initial Rank Ordering), a two-stage regression framework that is robust to outliers and heavy-tailed noise.

**Paper:** [arXiv:2602.14440](https://arxiv.org/abs/2602.14440)

## Why CAIRO?

Standard regression methods (MSE) have a fundamental problem: they **conflate learning ordering with learning scale**.

- **Ordering:** Which examples have higher/lower values?
- **Scale:** What are the actual numerical values?

When you have outliers or heavy-tailed distributions, MSE loss is dominated by extreme values. A single outlier can dramatically shift your model's predictions.

**CAIRO's key insight:** Learn ordering first (which is scale-invariant), then recover scale separately.

In [ ]:
import sys
sys.path.insert(0, '..')

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from models import CAIRO, create_cairo
from models.base import MLP

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## The Two-Stage Framework

### Stage 1: Learn Ranking via Scale-Invariant Loss

Instead of minimizing MSE, we minimize a **pairwise ranking loss**:

$$\mathcal{L}_{\text{rank}} = \sum_{i,j: y_i > y_j} w_{ij} \cdot \log(1 + e^{-\sigma(s_i - s_j)})$$

Where:
- $s_i = g(x_i)$ is the score for example $i$
- $w_{ij}$ is a weight (uniform for Kendall's τ, $|y_i - y_j|$ for Gini)
- $\sigma$ controls the sigmoid steepness

This loss penalizes misordered pairs: when $y_i > y_j$ but $s_i < s_j$.

**Why is this robust?** The log-sigmoid saturates for large score differences, so outliers can't dominate the gradient.

### Stage 2: Recover Scale via Isotonic Regression

After learning a scoring function $g(x)$ that correctly orders examples, we fit a **monotone increasing function** to map scores back to target values:

$$f^* = \arg\min_{f \in \mathcal{F}} \sum_i (y_i - f(g(x_i)))^2$$

Where $\mathcal{F}$ is the set of non-decreasing functions.

This is solved by **isotonic regression** (Pool Adjacent Violators Algorithm), which is:
- Non-parametric (no assumptions about the mapping)
- Efficient: O(n) on sorted data
- Guaranteed to preserve the ordering from Stage 1

## Creating Synthetic Data with Heavy Tails

Let's create data where CAIRO shines: heavy-tailed noise with outliers.

In [ ]:
def generate_heavy_tailed_data(n_samples=1000, n_features=10, noise_type='lognormal'):
    """
    Generate regression data with heavy-tailed noise.
    
    noise_type:
    - 'gaussian': Standard normal noise (for comparison)
    - 'lognormal': Heavy-tailed multiplicative noise
    - 'outliers': Gaussian with 10% outliers
    """
    # Features
    X = np.random.randn(n_samples, n_features)
    
    # True signal: linear with interaction
    weights = np.random.randn(n_features)
    signal = X @ weights / np.sqrt(n_features)
    
    # Apply noise
    if noise_type == 'gaussian':
        noise = np.random.randn(n_samples)
        y = signal + noise
    elif noise_type == 'lognormal':
        # Heteroskedastic, heavy-tailed noise
        mu = np.exp(signal)  # Mean depends on signal
        noise = np.random.lognormal(0, 1, n_samples) - 1  # Zero-mean lognormal
        y = mu + noise * np.sqrt(mu)  # Variance scales with mean
    elif noise_type == 'outliers':
        noise = np.random.randn(n_samples)
        # Add outliers: 10% of points have 10x noise
        outlier_mask = np.random.rand(n_samples) < 0.1
        noise[outlier_mask] *= 10
        y = signal + noise
    else:
        raise ValueError(f"Unknown noise_type: {noise_type}")
    
    return (
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32),
        signal
    )

# Generate different noise regimes
X_gaussian, y_gaussian, signal_gaussian = generate_heavy_tailed_data(noise_type='gaussian')
X_lognormal, y_lognormal, signal_lognormal = generate_heavy_tailed_data(noise_type='lognormal')
X_outliers, y_outliers, signal_outliers = generate_heavy_tailed_data(noise_type='outliers')

# Visualize the distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(y_gaussian.numpy(), bins=50, alpha=0.7)
axes[0].set_title('Gaussian Noise')
axes[0].set_xlabel('Target Value')

axes[1].hist(y_lognormal.numpy(), bins=50, alpha=0.7)
axes[1].set_title('Lognormal (Heavy-Tailed) Noise')
axes[1].set_xlabel('Target Value')

axes[2].hist(y_outliers.numpy(), bins=50, alpha=0.7)
axes[2].set_title('Gaussian + 10% Outliers')
axes[2].set_xlabel('Target Value')

plt.tight_layout()
plt.show()

## Understanding the Ranking Losses

CAIRO supports three ranking loss variants:

### 1. RankNet (Uniform Weights)
Optimizes **Kendall's τ** - treats all misordered pairs equally.

### 2. RankNet-GiniW (Absolute Gap Weights)
Optimizes **Gini covariance** - weights pairs by $|y_i - y_j|$, so large gaps matter more.

### 3. GiniNet-SoftRank (Pointwise)
Uses differentiable soft ranks - O(n log n) complexity vs O(n²) for pairwise.

Let's visualize how the log-sigmoid loss provides robustness:

In [ ]:
# Compare MSE vs RankNet loss gradients
score_diff = np.linspace(-5, 5, 100)

# MSE gradient (for a single pair): 2 * (y_i - y_j) - 2 * (s_i - s_j)
# Simplified: gradient w.r.t. score_diff when y_i > y_j
mse_gradient = -2 * np.ones_like(score_diff)  # Constant gradient

# RankNet gradient: sigmoid(-score_diff) when y_i > y_j
sigma = 1.0
ranknet_gradient = -1 / (1 + np.exp(sigma * score_diff))

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(score_diff, np.abs(mse_gradient), 'b-', linewidth=2, label='MSE Loss (constant)')
ax.plot(score_diff, np.abs(ranknet_gradient), 'r-', linewidth=2, label='RankNet Loss')

ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.fill_between(score_diff, 0, np.abs(ranknet_gradient), alpha=0.3, color='red')

ax.set_xlabel('Score Difference (s_i - s_j)', fontsize=12)
ax.set_ylabel('|Gradient|', fontsize=12)
ax.set_title('Gradient Magnitude: RankNet saturates, MSE does not', fontsize=14)
ax.legend(fontsize=11)
ax.set_ylim(0, 2.5)

ax.annotate('RankNet saturates:\nno big gradient from\nalready-correct pairs',
            xy=(3, 0.1), fontsize=10, ha='center')

plt.tight_layout()
plt.show()

## Training CAIRO

Let's train CAIRO and compare it to a standard MLP on heavy-tailed data.

In [ ]:
from scipy.stats import kendalltau, spearmanr

def train_cairo(X, y, n_epochs=100, lr=0.01):
    """Train CAIRO model."""
    model = CAIRO(
        d_in=X.shape[1],
        d_out=1,
        n_blocks=2,
        d_block=64,
        loss_type="ranknet",
    )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    losses = []
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        
        scores = model.get_scores(X)
        loss = model.compute_ranking_loss(scores, y)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    
    # Stage 2: Fit calibrator
    model.fit_calibrator(X, y)
    
    return model, losses

def train_mlp(X, y, n_epochs=100, lr=0.01):
    """Train standard MLP with MSE."""
    model = MLP(
        d_in=X.shape[1],
        d_out=1,
        n_blocks=2,
        d_block=64,
    )
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    losses = []
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        
        pred = model(X)
        loss = criterion(pred, y.unsqueeze(-1))
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    
    return model, losses

def evaluate_model(model, X, y, is_cairo=False):
    """Evaluate model on RMSE and ranking metrics."""
    model.eval()
    with torch.no_grad():
        pred = model(X).squeeze()
    
    rmse = torch.sqrt(((pred - y) ** 2).mean()).item()
    
    # Ranking metrics
    tau, _ = kendalltau(pred.numpy(), y.numpy())
    rho, _ = spearmanr(pred.numpy(), y.numpy())
    
    return {'rmse': rmse, 'kendall_tau': tau, 'spearman_rho': rho}

In [ ]:
# Train on heavy-tailed data
print("Training on Lognormal (heavy-tailed) data...")
cairo_model, cairo_losses = train_cairo(X_lognormal, y_lognormal)
mlp_model, mlp_losses = train_mlp(X_lognormal, y_lognormal)

# Evaluate
cairo_metrics = evaluate_model(cairo_model, X_lognormal, y_lognormal, is_cairo=True)
mlp_metrics = evaluate_model(mlp_model, X_lognormal, y_lognormal)

print(f"\nCAIRO - RMSE: {cairo_metrics['rmse']:.4f}, Kendall's τ: {cairo_metrics['kendall_tau']:.4f}")
print(f"MLP   - RMSE: {mlp_metrics['rmse']:.4f}, Kendall's τ: {mlp_metrics['kendall_tau']:.4f}")

In [ ]:
# Compare across noise regimes
results = {}

for name, (X, y) in [
    ('Gaussian', (X_gaussian, y_gaussian)),
    ('Lognormal', (X_lognormal, y_lognormal)),
    ('Outliers', (X_outliers, y_outliers)),
]:
    print(f"\n=== {name} Noise ===")
    
    cairo_m, _ = train_cairo(X, y)
    mlp_m, _ = train_mlp(X, y)
    
    cairo_res = evaluate_model(cairo_m, X, y, is_cairo=True)
    mlp_res = evaluate_model(mlp_m, X, y)
    
    results[name] = {'CAIRO': cairo_res, 'MLP': mlp_res}
    
    print(f"  CAIRO - RMSE: {cairo_res['rmse']:.4f}, τ: {cairo_res['kendall_tau']:.4f}")
    print(f"  MLP   - RMSE: {mlp_res['rmse']:.4f}, τ: {mlp_res['kendall_tau']:.4f}")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# RMSE comparison
noise_types = list(results.keys())
cairo_rmse = [results[n]['CAIRO']['rmse'] for n in noise_types]
mlp_rmse = [results[n]['MLP']['rmse'] for n in noise_types]

x = np.arange(len(noise_types))
width = 0.35

axes[0].bar(x - width/2, mlp_rmse, width, label='MLP (MSE)', color='blue', alpha=0.7)
axes[0].bar(x + width/2, cairo_rmse, width, label='CAIRO', color='red', alpha=0.7)
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE by Noise Type')
axes[0].set_xticks(x)
axes[0].set_xticklabels(noise_types)
axes[0].legend()

# Kendall's tau comparison
cairo_tau = [results[n]['CAIRO']['kendall_tau'] for n in noise_types]
mlp_tau = [results[n]['MLP']['kendall_tau'] for n in noise_types]

axes[1].bar(x - width/2, mlp_tau, width, label='MLP (MSE)', color='blue', alpha=0.7)
axes[1].bar(x + width/2, cairo_tau, width, label='CAIRO', color='red', alpha=0.7)
axes[1].set_ylabel("Kendall's τ")
axes[1].set_title('Ranking Accuracy by Noise Type')
axes[1].set_xticks(x)
axes[1].set_xticklabels(noise_types)
axes[1].legend()

plt.tight_layout()
plt.show()

## Visualizing the Two Stages

Let's see what Stage 1 (ranking) and Stage 2 (calibration) each contribute.

In [ ]:
# Train a CAIRO model and examine stages
cairo_demo, _ = train_cairo(X_lognormal, y_lognormal)

# Get Stage 1 scores (before calibration)
cairo_demo.eval()
with torch.no_grad():
    scores = cairo_demo.get_scores(X_lognormal).numpy()

# Get Stage 2 calibrated predictions
predictions = cairo_demo(X_lognormal).squeeze().numpy()
targets = y_lognormal.numpy()

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Stage 1: Scores vs Targets
axes[0].scatter(targets, scores, alpha=0.5, s=10)
axes[0].set_xlabel('True Target')
axes[0].set_ylabel('Stage 1 Score')
axes[0].set_title('Stage 1: Scores preserve ordering\n(not on target scale)')

# Stage 2 calibration function
# Sort by scores to show the isotonic function
sort_idx = np.argsort(scores)
axes[1].scatter(scores[sort_idx], targets[sort_idx], alpha=0.3, s=10, label='Data')
axes[1].plot(scores[sort_idx], predictions[sort_idx], 'r-', linewidth=2, label='Isotonic fit')
axes[1].set_xlabel('Stage 1 Score')
axes[1].set_ylabel('Target / Prediction')
axes[1].set_title('Stage 2: Isotonic regression\nmaps scores to target scale')
axes[1].legend()

# Final predictions vs true
axes[2].scatter(targets, predictions, alpha=0.5, s=10)
axes[2].plot([targets.min(), targets.max()], [targets.min(), targets.max()], 'r--', linewidth=2)
axes[2].set_xlabel('True Target')
axes[2].set_ylabel('CAIRO Prediction')
axes[2].set_title('Final Predictions\n(after calibration)')

plt.tight_layout()
plt.show()

## Key Takeaways

1. **CAIRO excels with heavy-tailed noise**: When your data has outliers or fat tails, CAIRO's ranking loss is more robust than MSE.

2. **Ranking vs Scale**: The two-stage approach separates concerns - first get the ordering right, then recover scale.

3. **Log-sigmoid saturation**: The pairwise loss saturates for large errors, naturally limiting outlier influence.

4. **Use cases**:
   - Financial data (fat-tailed returns)
   - Heteroskedastic targets (variance changes with level)
   - Data with measurement outliers
   - When ranking matters more than exact values

5. **Trade-offs**:
   - On clean Gaussian data, MSE may be slightly better
   - O(n²) complexity for pairwise losses (use softrank for large batches)
   - Requires two-stage training